In [ ]:
import backtrader as bt
import pandas as pd

btc_csv_parsed = bt.feeds.GenericCSVData(
    dataname="../data/BTCUSDT_2025-06-03_UTC_with_lookback.csv",
    dtformat=('%Y-%m-%d %H:%M:%S'),
    datetime=0,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=-1
)

### Testing code

In [18]:
import backtrader as bt
import pandas as pd

data = bt.feeds.GenericCSVData(
    dataname="../data/BTCUSDT_2025-06-02_UTC_with_lookback.csv",
    dtformat='%Y-%m-%d %H:%M:%S',
    timeframe=bt.TimeFrame.Minutes,
    compression=15,
    datetime=0,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=-1  # No open interest in your CSV
)

class EmaCrossover(bt.Strategy):
    params = (
        ('short_period', 9),
        ('long_period', 21),
    )

    def __init__(self):
        self.short_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.short_period)
        self.long_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.long_period)
        
        # Use Backtrader's built-in CrossOver indicator
        self.crossover = bt.indicators.CrossOver(self.short_ema, self.long_ema)

        # Initialize counters
        self.bullish_crossovers = 0
        self.bearish_crossovers = 0
        self.total_crossovers = 0

    def next(self):

        if self.crossover > 0:  # Bullish crossover
            self.bullish_crossovers += 1
            self.total_crossovers += 1
            print(f"🟢 BULLISH CROSSOVER #{self.bullish_crossovers} on {self.data.datetime.date(0)} --- {self.short_ema[0]} vs {self.long_ema[0]}")
            
        elif self.crossover < 0:  # Bearish crossover
            self.bearish_crossovers += 1
            self.total_crossovers += 1
            print(
                f"🔴 BEARISH CROSSOVER #{self.bearish_crossovers} on {self.data.datetime.date(0)}  --- {self.short_ema[0]} vs {self.long_ema[0]} ")
    
       


cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(EmaCrossover)

cerebro.run()

🔴 BEARISH CROSSOVER #1 on 2025-06-02  --- 105262.18770801499 vs 105285.68755086034 
🟢 BULLISH CROSSOVER #1 on 2025-06-02 --- 104996.17241110405 vs 104989.81128619946
🔴 BEARISH CROSSOVER #2 on 2025-06-02  --- 105014.95482964441 vs 105074.17642160201 
🟢 BULLISH CROSSOVER #2 on 2025-06-02 --- 104309.25054745017 vs 104275.40824295621
🔴 BEARISH CROSSOVER #3 on 2025-06-02  --- 104254.2357313885 vs 104262.23678459381 
🟢 BULLISH CROSSOVER #3 on 2025-06-02 --- 104276.6685851108 vs 104271.70616781255


## Strategy

In [6]:
from datetime import datetime, timedelta
import numpy as np

class EMACrossoverStrategy(bt.Strategy):
    """
    EMA Crossover Strategy with dynamic parameters
    Always maintains a position - switches between long and short
    """

    params = (
        ('fast_ema_period', 9),
        ('slow_ema_period', 21),
        ('amount', 1000),
        ('leverage', 1),
        ('printlog', True),
    )

    def __init__(self):
        # Calculate EMAs
        self.fast_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.fast_ema_period
        )
        self.slow_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.slow_ema_period
        )
        
        # Crossover signals
        self.crossover = bt.indicators.CrossOver(self.fast_ema, self.slow_ema)
        
        # Track position state
        self.current_position = None  # 'long', 'short', or None
        
        # Trade tracking
        self.trade_count = 0
        self.winning_trades = 0
        self.losing_trades = 0
        self.total_pnl = 0
        
        # Store trade details
        self.trades_log = []

    def log(self, txt, dt=None):
        """Logging function"""
        if self.params.printlog:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}: {txt}')

    def next(self):
        # Skip if we don't have enough data for EMAs
        if len(self.data) < max(self.params.fast_ema_period, self.params.slow_ema_period):
            return

        current_price = self.data.close[0]

        # Check for crossover signals
        if self.crossover[0] > 0:  # Fast EMA crosses above Slow EMA (Bullish)
            if self.current_position == 'short':
                # Close short position
                self.close()
                self.log(f'CLOSE SHORT at {current_price:.2f}')

            # Open long position
            size = self.calculate_position_size()
            self.buy(size=size)
            self.current_position = 'long'
            self.log(f'OPEN LONG at {current_price:.2f}, Size: {size}')

        # Fast EMA crosses below Slow EMA (Bearish)
        elif self.crossover[0] < 0:
            if self.current_position == 'long':
                # Close long position
                self.close()
                self.log(f'CLOSE LONG at {current_price:.2f}')

            # Open short position
            size = self.calculate_position_size()
            self.sell(size=size)
            self.current_position = 'short'
            self.log(f'OPEN SHORT at {current_price:.2f}, Size: {size}')

    def calculate_position_size(self):
        """Calculate position size based on amount and leverage"""
        current_price = self.data.close[0]
        # Calculate size based on available cash and leverage
        cash = self.broker.get_cash()
        max_position_value = min(
            self.params.amount * self.params.leverage, cash)
        size = max_position_value / current_price
        return size

    def notify_trade(self, trade):
        """Called when a trade is closed"""
        if not trade.isclosed:
            return

        self.trade_count += 1
        pnl = trade.pnl
        self.total_pnl += pnl

        if pnl >= 0:
            self.winning_trades += 1
            status = 'WIN'
        else:
            self.losing_trades += 1
            status = 'LOSS'

        # Log trade details
        trade_info = {
            'trade_num': self.trade_count,
            'entry_date': bt.num2date(trade.dtopen).strftime('%Y-%m-%d %H:%M:%S'),
            'exit_date': bt.num2date(trade.dtclose).strftime('%Y-%m-%d %H:%M:%S'),
            'entry_price': trade.price,
            'exit_price': trade.pnlcomm / trade.size + trade.price if trade.size != 0 else 0,
            'size': trade.size,
            'pnl': pnl,
            'pnl_percent': (pnl / abs(trade.price * trade.size)) * 100 if trade.size != 0 else 0,
            'status': status
        }

        self.trades_log.append(trade_info)

        self.log(
            f'TRADE #{self.trade_count} - {status}: PnL: {pnl:.2f} ({trade_info["pnl_percent"]:.2f}%)')

In [7]:
import backtrader as bt


def run_ema_backtest(csv_file, fast_ema=9, slow_ema=21, amount=1000,
                     leverage=1, initial_cash=10000):
    """
    Simple EMA crossover backtest function
    
    Parameters:
    - csv_file: path to your CSV file
    - fast_ema: fast EMA period (default 9)
    - slow_ema: slow EMA period (default 21)  
    - amount: position size
    - leverage: trading leverage
    - initial_cash: starting cash
    """

    print(f"Starting backtest with EMA({fast_ema}) x EMA({slow_ema})")

    # Create Cerebro engine
    cerebro = bt.Cerebro()

    # Add your strategy (assuming you named it EMACrossoverStrategy)
    cerebro.addstrategy(
        EMACrossoverStrategy,
        fast_ema_period=fast_ema,
        slow_ema_period=slow_ema,
        amount=amount,
        leverage=leverage
    )

    # Add your data feed (same way you're doing it)
    data = bt.feeds.GenericCSVData(
        dataname=csv_file,
        dtformat=('%Y-%m-%d %H:%M:%S'),  # Adjust format for 15min data
        datetime=0,
        open=1,
        high=2,
        low=3,
        close=4,
        volume=5,
        openinterest=-1
    )
    cerebro.adddata(data)

    # Set broker
    cerebro.broker.setcash(initial_cash)
    cerebro.broker.setcommission(commission=0.001)  # 0.1% commission

    # Store starting value
    start_value = cerebro.broker.getvalue()
    print(f'Starting Portfolio Value: ${start_value:,.2f}')

    # Run backtest
    results = cerebro.run()
    strategy = results[0]

    # Get final value
    final_value = cerebro.broker.getvalue()

    # Calculate results
    total_return = ((final_value - start_value) / start_value) * 100

    # Print results
    print("\n" + "="*50)
    print("BACKTEST RESULTS")
    print("="*50)
    print(f"Starting Value: ${start_value:,.2f}")
    print(f"Final Value: ${final_value:,.2f}")
    print(f"Total Return: {total_return:+.2f}%")
    print(f"Total P&L: ${final_value - start_value:+,.2f}")
    print(f"Total Trades: {strategy.trade_count}")
    print(f"Winning Trades: {strategy.winning_trades}")
    print(f"Losing Trades: {strategy.losing_trades}")

    if strategy.trade_count > 0:
        win_rate = (strategy.winning_trades / strategy.trade_count) * 100
        print(f"Win Rate: {win_rate:.1f}%")

    print("="*50)

    return {
        'start_value': start_value,
        'final_value': final_value,
        'total_return': total_return,
        'total_trades': strategy.trade_count,
        'win_rate': win_rate if strategy.trade_count > 0 else 0
    }


# Example usage:
if __name__ == "__main__":
    # Run backtest with your CSV file
    results = run_ema_backtest(
        csv_file="../data/BTCUSDT_2025-06-04_15min.csv",
        fast_ema=9,
        slow_ema=21,
        amount=1000,
        leverage=1,
        initial_cash=10000
    )

    print(f"\nFinal Return: {results['total_return']:.2f}%")

Starting backtest with EMA(9) x EMA(21)
Starting Portfolio Value: $10,000.00
2025-06-03: OPEN LONG at 105321.30, Size: 0.009494755571759939
2025-06-03: CLOSE LONG at 105226.80
2025-06-03: OPEN SHORT at 105226.80, Size: 0.009503282433752617
2025-06-03: CLOSE SHORT at 105259.00
2025-06-03: OPEN LONG at 105259.00, Size: 0.00950037526482296
2025-06-03: CLOSE LONG at 105689.80
2025-06-03: OPEN SHORT at 105689.80, Size: 0.009461650982403222
2025-06-03: CLOSE SHORT at 105960.00
2025-06-03: OPEN LONG at 105960.00, Size: 0.009437523593808984
2025-06-03: CLOSE LONG at 105755.30
2025-06-03: OPEN SHORT at 105755.30, Size: 0.009455790868164527
2025-06-04: TRADE #1 - WIN: PnL: 0.00 (0.00%)
2025-06-04: TRADE #2 - WIN: PnL: 0.00 (0.00%)

BACKTEST RESULTS
Starting Value: $10,000.00
Final Value: $9,994.01
Total Return: -0.06%
Total P&L: $-5.99
Total Trades: 2
Winning Trades: 2
Losing Trades: 0
Win Rate: 100.0%

Final Return: -0.06%
